# Calculate biggest donors for County Executive candidates currently in office

In [1]:
import glob
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
CONTRIB_REGEX = r'Full Name of Contrib.*\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(\d+)\n(\d+)\n(\d+)\n'
RECEIPT_REGEX = r'Full Name \n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(.*)\n(\d+)\n(\d+)\n(\d+)\nReceipt Description\n.*\n'
UNITEMIZED_REGEX = r'Unitemized  Contributions Received - \$ 50.00 or Less Per Contributor\nTOTAL for the Reporting Period           \(1\)\n\$\n(.*)\n'

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_contributions(name):
    contributions = []
    for file in glob.glob('input/%s_*.pdf' % name.capitalize()):
        contributions.append(flatten(process_report(file)))

    df = pd.DataFrame(
        flatten(contributions),
        columns=['name', 'address1', 'city', 'state', 'zipcode', 'amount', 'month', 'day', 'year'])
    df['name'] = df.name.str.upper()
    df['amount'] = pd.to_numeric(df.amount.str.replace(',', ''))

    return df

def process_report(filename):
    reader = PdfReader(filename)
    text = ""

    for page in reader.pages:
        text += page.extract_text() + "\n"

    unitemized = re.findall(UNITEMIZED_REGEX, text)
    if len(unitemized) == 0:
        unitemized = []
    else:
        unitemized = [(
            'UNITEMIZED',
            '',
            'PITTSBURGH',
            'PA',
            '',
            unitemized[0],
            '',
            '',
            ''
        )]

    return [
        re.findall(CONTRIB_REGEX, text),
        re.findall(RECEIPT_REGEX, text),
        unitemized
    ]

### John Weinstein (state committee)

In [3]:
df_weinstein = get_contributions('Weinstein')

df_weinstein['name'] = df_weinstein.name.replace({
    "AMALGAMATED TRANSIT UNION COPE": "ATU COPE VOLUNTARY ACCOUNT",
    "LABORERS DISTRICT COUNCIL OF WESTERN PENNSYLVANIA": "WESTERN PENNSYLVANIA LABORERS",
    "LABORERS' DISTRICT COUNCIL OF WESTERN PENNSYLVANIA": "WESTERN PENNSYLVANIA LABORERS",
    "MIDATLANTIC POLITICAL LEAGUE - MALPA": "MID-ATLANTIC LABORERS' POLITICAL LEAGUE",
    "STEAMFITTERS LOCAL UNION 449 PAC FUND": "STEAMFITTERS LOCAL UNON #449",
    "WESTERN PA LABORERS UNION PAC": "WESTERN PENNSYLVANIA LABORERS",
    "WESTERN PENNSYLVANIA LABORERS' PAC": "WESTERN PENNSYLVANIA LABORERS"
})

In [14]:
df_weinstein_topcontribs = df_weinstein.groupby('name').amount.sum().to_frame()

weinstein_sum = df_weinstein.amount.sum()
df_weinstein_topcontribs['pct'] = df_weinstein_topcontribs.amount / weinstein_sum

df_weinstein_topcontribs.sort_values(by='amount', ascending=False).head(10)

,amount,pct
name,,
WESTERN PENNSYLVANIA LABORERS,90000.0,0.180471
ATU COPE VOLUNTARY ACCOUNT,37614.0,0.075425
FNB CORPORATION PAC,22500.0,0.045118
MID-ATLANTIC LABORERS' POLITICAL LEAGUE,22000.0,0.044115
FRIENDS OF RANDY MARTINI,20000.0,0.040105
STEAMFITTERS LOCAL UNON #449,15000.0,0.030078
IBEW LOCAL UNION NO. 5 PAC,10000.0,0.020052
AMALGAMATED TRANSIT UNION - LOCAL 85,10000.0,0.020052
JAMES P. GRANT,10000.0,0.020052


### Sara Innamorato (state committee)

In [5]:
df_innamorato = get_contributions('Innamorato')

df_innamorato['name'] = df_innamorato.name.replace({
    "1776 PAC": "1776 PAC (UFCW)",
    "AFSCME COUNCIL 13 POL & LEG ACCT": "AFSCME COUNCIL 13",
    "EVAN SEGAL": "EVAN J. SEGAL",
    "PENNSYLVANIA SEIU COPE": "SEIU HEALTHCARE PA COPE",
    "PFT POL ACTION FUND (PGH FED TEACHERS)": "PFT POLITICAL ACTION FUND",
    "TEAMSERS LOCAL UNION 249": "TEAMSTERS LOCAL UNION 249 - DRIVE FUND",
    "TEAMSTERS LOCAL UNION 249 DRIVE FUND": "TEAMSTERS LOCAL UNION 249 - DRIVE FUND",
    "WESTERN PENNSYLVANIA LABORERS POLITICAL ACTION FUND": "WESTERN PENNSYLVANIA LABORERS",
})

In [6]:
df_innamorato_topcontribs = df_innamorato.groupby('name').amount.sum().to_frame()

innamorato_sum = df_innamorato.amount.sum()
df_innamorato_topcontribs['pct'] = df_innamorato_topcontribs.amount / innamorato_sum

df_innamorato_topcontribs.sort_values(by='amount', ascending=False).head(10)

,amount,pct
name,,
UNITEMIZED,35293.34,0.109579
EVAN J. SEGAL,16020.63,0.049741
SEIU HEALTHCARE PA COPE,11250.00,0.034929
PENNSYLVANIA SIERRA CLUB PAC,10000.00,0.031048
DAVID TURNER,7650.00,0.023752
PENNSYLVANIA DEMOCRATIC PARTY,7613.90,0.023640
NANCY BERNSTEIN,6500.00,0.020181
WOMEN FOR THE FUTURE,6500.00,0.020181
ALLEGHENY COUNTY DEMOCRATIC DELEGATION,5750.00,0.017853


### Michael Lamb (state and local committees)

In [7]:
namespre22 = [
    'filer_id', 'election_year', 'cycle',
    'name', 'address1', 'address2', 'city', 'state', 'zipcode',
    'date', 'amount', 'description'
]

names2022 = [
    'report_id', 'filer_id', 'election_year', 'filer_date', 'cycle',
    'name', 'address1', 'address2', 'city', 'state', 'zipcode',
    'date', 'amount', 'description'
]

expenses = pd.concat([
    pd.read_csv('input/state_expense_2017.csv', encoding='unicode_escape', names=namespre22),
    pd.read_csv('input/state_expense_2018.csv', encoding='unicode_escape', names=namespre22, skiprows=[1333, 22823, 62725, 108098]),
    pd.read_csv('input/state_expense_2019.csv', encoding='unicode_escape', names=namespre22),
    pd.read_csv('input/state_expense_2020.csv', encoding='unicode_escape', names=namespre22, skiprows=[60172]),
    pd.read_csv('input/state_expense_2021.csv', encoding='unicode_escape', names=namespre22, skiprows=[14403, 66485, 66486, 66487, 66488, 66489, 66490, 66491, 87279]),
    pd.read_csv('input/state_expense_2022.csv', encoding='unicode_escape', usecols=names2022, names=names2022)
])

expenses.loc[expenses.description.isna(), 'description'] = ''
expenses.loc[expenses.name.isna(), 'name'] = ''

expenses['description'] = expenses.description.str.upper()
expenses['name'] = expenses.name.str.upper()
expenses['address1'] = expenses.address1.str.upper()

### Remove malformated vendor
expenses = expenses.drop(expenses[expenses['name'] == 'NCH SOFTWARE'].index)

In [8]:
df_lamb = expenses[
    (
        ~expenses.name.isin([
            'FRIENDS OF MICHAEL LAMBERT',
            'FRIENDS OF RICH FITZGERALD'
            'LAW OFFICE OF MICHAEL LAMBERT LLC',
            'MARINE CORPS LEAGUE',
        ])
    ) & (
        (expenses.name.str.contains('MIKE LAMB') |
        expenses.name.str.contains('LAMB FOR PA') |
        expenses.name.str.contains('MICHAEL LAMB'))
    ) | (
        expenses.address1.notna() & (expenses.address1.str.contains(' BOX 1835') | expenses.address1.str.contains('1015 GRANDVIEW'))
    )
]

In [10]:
df_lamb_topcontribs = df_lamb.groupby('filer_id').amount.sum().to_frame()
df_lamb_topcontribs = pd.merge(
    df_lamb_topcontribs,
    pd.read_csv('input/committees.csv', dtype={'Committee Number': 'str'}, usecols=['Committee Number', 'Committee Name']),
    left_on='filer_id',
    right_on='Committee Number')

lamb_sum = df_lamb.amount.sum()
df_lamb_topcontribs['pct'] = df_lamb_topcontribs.amount / lamb_sum

df_lamb_topcontribs[['Committee Number', 'Committee Name', 'amount', 'pct']].sort_values(by='amount', ascending=False).head(10)

,Committee Number,Committee Name,amount,pct
59,8400326,LOCAL 0005 IBEW PAC,56250.0,0.142927
51,8000488,LOCAL 0027 PLUMBERS UNION PAC,48550.0,0.123362
64,8600238,LOCAL 0449 STEAMFITTERS UNION PAC,40350.0,0.102526
62,8600169,LOCAL 0066 PAC CLUB,37000.0,0.094014
46,7900296,LOCAL 0690 PLUMBERS UNION POL ACTION FUND,27500.0,0.069875
54,8100217,LOCAL 0542 IUOE OPER ENG (NORRISTN),25000.0,0.063523
82,9500250,GREATER PA CARPENTERS PAC,18050.0,0.045864
58,8200660,INTL UNION PAINTERS ALLIED TRADE,14000.0,0.035573
39,20190173,WESTERN PENNSYLVANIA LABORERS 2019 PAC,10000.0,0.025409
71,9300158,INTL BROTHERHOOD BOILERMAKERS 154,7100.0,0.018041


### Liv Bennett (local committee)